In [1]:
import pandas as pd

df = pd.read_csv("../data/cancer_prepared.csv")

In [2]:
df.head()

,Age_bin_1,Age_bin_2,Age_bin_3,Genetic_Risk_bin_1,Genetic_Risk_bin_2,Genetic_Risk_bin_3,Air_Pollution_bin_1,Air_Pollution_bin_2,Air_Pollution_bin_3,Alcohol_Use_bin_1,...,Cancer_Type=Leukemia,Cancer_Type=Liver,Cancer_Type=Lung,Cancer_Type=Prostate,Cancer_Type=Skin,Cancer_Stage=Stage 0,Cancer_Stage=Stage I,Cancer_Stage=Stage II,Cancer_Stage=Stage III,Cancer_Stage=Stage IV
0,False,False,True,False,True,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1,True,False,False,True,False,False,False,True,False,False,...,True,False,False,False,False,True,False,False,False,False
2,False,False,True,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,True,False,False
3,True,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,True,False,False,False,True,False,True,False,False,True,...,False,False,False,False,True,False,False,False,True,False


In [3]:
from src.utils import get_and_show_concepts, get_rules_for_rounds, print_top_conf1_rules_by_support, \
    evaluate_itemset_across_rounds, get_and_show_rules, analyze_rule_evolution

In [4]:
get_and_show_concepts(df.head(2000), 0.15, 2)

1: ['Smoking_bin_1', 'Target_Severity_Score_bin_1'] - Support: 0.188
2: ['Genetic_Risk_bin_1', 'Target_Severity_Score_bin_1'] - Support: 0.185
3: ['Air_Pollution_bin_1', 'Target_Severity_Score_bin_1'] - Support: 0.1845
4: ['Genetic_Risk_bin_3', 'Target_Severity_Score_bin_3'] - Support: 0.1825
5: ['Smoking_bin_3', 'Target_Severity_Score_bin_3'] - Support: 0.18
6: ['Alcohol_Use_bin_1', 'Target_Severity_Score_bin_1'] - Support: 0.1675
7: ['Obesity_Level_bin_1', 'Target_Severity_Score_bin_1'] - Support: 0.1635
8: ['Alcohol_Use_bin_3', 'Target_Severity_Score_bin_3'] - Support: 0.1575
9: ['Air_Pollution_bin_3', 'Target_Severity_Score_bin_3'] - Support: 0.154


In [5]:
get_and_show_rules(df, min_support=0.15, min_confidence=0.3)

1: Smoking_bin_1 -> Target_Severity_Score_bin_1
Support: 0.1859 - Confidence: 0.5525 - Lift: 1.6436
2: Genetic_Risk_bin_1 -> Target_Severity_Score_bin_1
Support: 0.1846 - Confidence: 0.5524 - Lift: 1.6431
3: Smoking_bin_3 -> Target_Severity_Score_bin_3
Support: 0.1840 - Confidence: 0.5530 - Lift: 1.6672
4: Genetic_Risk_bin_3 -> Target_Severity_Score_bin_3
Support: 0.1783 - Confidence: 0.5486 - Lift: 1.6537
5: Air_Pollution_bin_1 -> Target_Severity_Score_bin_1
Support: 0.1689 - Confidence: 0.4929 - Lift: 1.4662
6: Alcohol_Use_bin_1 -> Target_Severity_Score_bin_1
Support: 0.1657 - Confidence: 0.4940 - Lift: 1.4696
7: Alcohol_Use_bin_3 -> Target_Severity_Score_bin_3
Support: 0.1609 - Confidence: 0.4914 - Lift: 1.4814
8: Air_Pollution_bin_3 -> Target_Severity_Score_bin_3
Support: 0.1581 - Confidence: 0.4889 - Lift: 1.4737


ESS Dataset

In [6]:
df = pd.read_csv("../data/essround1.csv")

In [7]:
df.head()

,aesfdrk_1_2,aesfdrk_2_2,clsprty_1_3,clsprty_2_3,clsprty_3_3,freehms_1_3,freehms_2_3,freehms_3_3,gincdif_1_3,gincdif_2_3,...,trstplc_3_3,trstplt_1_3,trstplt_2_3,trstplt_3_3,trstprl_1_3,trstprl_2_3,trstprl_3_3,trstun_1_3,trstun_2_3,trstun_3_3
0,False,False,True,False,False,False,False,False,False,False,...,True,False,False,True,False,False,True,False,False,False
1,True,False,True,False,False,False,True,False,False,False,...,True,False,True,False,False,False,True,False,True,False
2,True,False,True,False,False,False,True,False,False,False,...,True,False,True,False,False,False,True,False,False,True
3,False,True,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,False,True
4,False,True,True,False,False,True,False,False,True,False,...,True,False,False,True,False,False,True,False,True,False


In [8]:
get_and_show_concepts(df.head(20), 0.7, 2)

1: ['clsprty_1_3', 'impcntr_1_3'] - Support: 0.9
2: ['health_1_3', 'impcntr_1_3'] - Support: 0.85
3: ['impcntr_1_3', 'impfree_1_3'] - Support: 0.85
4: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3'] - Support: 0.8
5: ['clsprty_1_3', 'impcntr_1_3', 'impfree_1_3'] - Support: 0.8
6: ['impcntr_1_3', 'stfhlth_3_3'] - Support: 0.8
7: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3', 'iplylfr_1_3'] - Support: 0.75
8: ['clsprty_1_3', 'impcntr_1_3', 'stfhlth_3_3'] - Support: 0.75
9: ['happy_3_3', 'impcntr_1_3'] - Support: 0.75
10: ['health_1_3', 'impcntr_1_3', 'impfree_1_3'] - Support: 0.75
11: ['health_1_3', 'impcntr_1_3', 'stfhlth_3_3'] - Support: 0.75
12: ['impcntr_1_3', 'ipeqopt_1_3'] - Support: 0.75
13: ['clsprty_1_3', 'happy_3_3', 'impcntr_1_3'] - Support: 0.7
14: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3', 'impfree_1_3'] - Support: 0.7
15: ['clsprty_1_3', 'health_1_3', 'impcntr_1_3', 'stfhlth_3_3'] - Support: 0.7
16: ['clsprty_1_3', 'impcntr_1_3', 'ipeqopt_1_3'] - Support: 0.7
17: ['happy_3_3',

In [9]:
get_and_show_rules(df, min_support=0.4, min_confidence=0.99)

1: impcntr_1_3, happy_3_3, aesfdrk_2_2, iplylfr_1_3, health_1_3 -> clsprty_1_3
Support: 0.4402 - Confidence: 0.9905 - Lift: 1.0151
2: prtdgcl_1_3 -> clsprty_1_3
Support: 0.4081 - Confidence: 1.0000 - Lift: 1.0248


Implications for 9 rounds

In [10]:
get_rules_for_rounds(min_support=0.7, min_confidence=1)


================= Round #1 =================

================= Round #2 =================

================= Round #3 =================

================= Round #4 =================

================= Round #5 =================

================= Round #6 =================

================= Round #7 =================

================= Round #8 =================
1: ipeqopt_1_3 -> clsprty_1_3
Support: 0.7452 - Confidence: 1.0000 - Lift: 1.0006

================= Round #9 =================

==== Rules that appear in 2 or more rounds ====



In [11]:
get_rules_for_rounds(min_support=0.6, min_confidence=1)


================= Round #1 =================

================= Round #2 =================

================= Round #3 =================
1: freehms_1_3, stfhlth_3_3 -> clsprty_1_3
Support: 0.6285 - Confidence: 1.0000 - Lift: 1.0067

================= Round #4 =================

================= Round #5 =================

================= Round #6 =================

================= Round #7 =================

================= Round #8 =================
1: ipeqopt_1_3 -> clsprty_1_3
Support: 0.7452 - Confidence: 1.0000 - Lift: 1.0006
2: iplylfr_1_3, ipeqopt_1_3 -> clsprty_1_3
Support: 0.6976 - Confidence: 1.0000 - Lift: 1.0006
3: ipeqopt_1_3, impcntr_1_3 -> clsprty_1_3
Support: 0.6795 - Confidence: 1.0000 - Lift: 1.0006
4: ipudrst_1_3 -> clsprty_1_3
Support: 0.6767 - Confidence: 1.0000 - Lift: 1.0006
5: freehms_1_3, ipeqopt_1_3 -> clsprty_1_3
Support: 0.6518 - Confidence: 1.0000 - Lift: 1.0006
6: ipeqopt_1_3, happy_3_3 -> clsprty_1_3
Support: 0.6484 - Confidence: 1.0000 - Lift: 1.

In [12]:
get_rules_for_rounds(min_support=0.5, min_confidence=1)


================= Round #1 =================

================= Round #2 =================

================= Round #3 =================
1: freehms_1_3, stfhlth_3_3 -> clsprty_1_3
Support: 0.6285 - Confidence: 1.0000 - Lift: 1.0067
2: iplylfr_1_3, freehms_1_3, stfhlth_3_3 -> clsprty_1_3
Support: 0.5684 - Confidence: 1.0000 - Lift: 1.0067
3: freehms_1_3, stfhlth_3_3, happy_3_3 -> clsprty_1_3
Support: 0.5567 - Confidence: 1.0000 - Lift: 1.0067
4: freehms_1_3, stfhlth_3_3, impcntr_1_3 -> clsprty_1_3
Support: 0.5551 - Confidence: 1.0000 - Lift: 1.0067
5: prtdgcl_1_3 -> clsprty_1_3
Support: 0.5350 - Confidence: 1.0000 - Lift: 1.0067
6: freehms_1_3, stfhlth_3_3, stflife_3_3 -> clsprty_1_3
Support: 0.5306 - Confidence: 1.0000 - Lift: 1.0067
7: freehms_1_3, aesfdrk_2_2, stfhlth_3_3 -> clsprty_1_3
Support: 0.5239 - Confidence: 1.0000 - Lift: 1.0067
8: stflife_3_3, freehms_1_3, stfhlth_3_3, happy_3_3 -> clsprty_1_3
Support: 0.5122 - Confidence: 1.0000 - Lift: 1.0067
9: freehms_1_3, ipeqopt_1_3,

In [ ]:
print_top_conf1_rules_by_support(3)  #+- 15 minutes to find


================= Round #1 =================
1: prtdgcl_1_3 -> clsprty_1_3
Support: 0.4081 - Confidence: 1.0000 - Lift: 1.0248
2: prtdgcl_1_3, impcntr_1_3 -> clsprty_1_3
Support: 0.3612 - Confidence: 1.0000 - Lift: 1.0248
3: prtdgcl_1_3, iplylfr_1_3 -> clsprty_1_3
Support: 0.3586 - Confidence: 1.0000 - Lift: 1.0248

================= Round #2 =================
1: prtdgcl_1_3 -> clsprty_1_3
Support: 0.4882 - Confidence: 1.0000 - Lift: 1.0137
2: prtdgcl_1_3, iplylfr_1_3 -> clsprty_1_3
Support: 0.4274 - Confidence: 1.0000 - Lift: 1.0137
3: prtdgcl_1_3, happy_3_3 -> clsprty_1_3
Support: 0.4235 - Confidence: 1.0000 - Lift: 1.0137

================= Round #3 =================
1: freehms_1_3, stfhlth_3_3 -> clsprty_1_3
Support: 0.6285 - Confidence: 1.0000 - Lift: 1.0067
2: iplylfr_1_3, freehms_1_3, stfhlth_3_3 -> clsprty_1_3
Support: 0.5684 - Confidence: 1.0000 - Lift: 1.0067
3: freehms_1_3, stfhlth_3_3, happy_3_3 -> clsprty_1_3
Support: 0.5567 - Confidence: 1.0000 - Lift: 1.0067

==========

In [18]:
evaluate_itemset_across_rounds(base={"ipeqopt_1_3"}, add={"clsprty_1_3"})

 {'ipeqopt_1_3'} → {'clsprty_1_3'} 
Round 1:
  Support : 0.7346
  Confidence : 0.9776
  Lift : 1.0018
Round 2:
  Support : 0.7745
  Confidence : 0.9878
  Lift : 1.0013
Round 3:
  Support : 0.7848
  Confidence : 0.9937
  Lift : 1.0003
Round 4:
  Support : 0.7568
  Confidence : 0.9903
  Lift : 0.9983
Round 5:
  Support : 0.7694
  Confidence : 0.9939
  Lift : 0.9992
Round 6:
  Support : 0.7838
  Confidence : 0.9973
  Lift : 1.0026
Round 7:
  Support : 0.7609
  Confidence : 0.9963
  Lift : 1.0014
Round 8:
  Support : 0.7452
  Confidence : 1.0000
  Lift : 1.0006
Round 9:
  Support : 0.7408
  Confidence : 0.9909
  Lift : 0.9977


In [26]:
evaluate_itemset_across_rounds(base={"ipeqopt_1_3", "stflife_3_3", "impcntr_1_3", "iplylfr_1_3", "happy_3_3"},
                               add={"clsprty_1_3"})

 {'impcntr_1_3', 'stflife_3_3', 'ipeqopt_1_3', 'happy_3_3', 'iplylfr_1_3'} → {'clsprty_1_3'} 
Round 1:
  Support : 0.4286
  Confidence : 0.9855
  Lift : 1.0099
Round 2:
  Support : 0.4370
  Confidence : 0.9911
  Lift : 1.0046
Round 3:
  Support : 0.4828
  Confidence : 0.9954
  Lift : 1.0021
Round 4:
  Support : 0.4722
  Confidence : 0.9916
  Lift : 0.9996
Round 5:
  Support : 0.4748
  Confidence : 0.9926
  Lift : 0.9979
Round 6:
  Support : 0.5035
  Confidence : 0.9979
  Lift : 1.0032
Round 7:
  Support : 0.4737
  Confidence : 0.9952
  Lift : 1.0003
Round 8:
  Support : 0.5062
  Confidence : 1.0000
  Lift : 1.0006
Round 9:
  Support : 0.5116
  Confidence : 0.9901
  Lift : 0.9969


In [19]:
evaluate_itemset_across_rounds(base={"freehms_1_3", "ipeqopt_1_3", "stfhlth_3_3"},
                               add={"clsprty_1_3"})

 {'stfhlth_3_3', 'freehms_1_3', 'ipeqopt_1_3'} → {'clsprty_1_3'} 
Round 1:
  Support : 0.3955
  Confidence : 0.9791
  Lift : 1.0034
Round 2:
  Support : 0.4685
  Confidence : 0.9870
  Lift : 1.0005
Round 3:
  Support : 0.5100
  Confidence : 1.0000
  Lift : 1.0067
Round 4:
  Support : 0.5284
  Confidence : 0.9936
  Lift : 1.0016
Round 5:
  Support : 0.5581
  Confidence : 0.9948
  Lift : 1.0001
Round 6:
  Support : 0.5757
  Confidence : 0.9972
  Lift : 1.0026
Round 7:
  Support : 0.5376
  Confidence : 0.9958
  Lift : 1.0009
Round 8:
  Support : 0.5294
  Confidence : 1.0000
  Lift : 1.0006
Round 9:
  Support : 0.4986
  Confidence : 0.9921
  Lift : 0.9989


In [20]:
analyze_rule_evolution(round=8, num_of_rules=10, min_support=0.6, min_confidence=1)

NameError: name 'analyze_rule_evolution' is not defined

Work with output(change to better names of the columns)

Lift tells us whether A and B happen together more often than expected if they were independent.